In [108]:
import  requests
import pandas as pd

In [109]:
bike_stations_df = pd.read_csv('bike_stations.csv')

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [112]:

# Foursquare API details
foursquare_api_key = "fsq3KxiUeOhGI4zSxgEr+yscwLw5cHupSA6OUnYtfizERgY="
foursquare_url = "https://api.foursquare.com/v3/places/search"

# Function to send a request for a single bike station
def query_foursquare(lat, lon, radius=1000):
    headers = {"Authorization": foursquare_api_key}
    params = {
        "ll": f"{lat},{lon}",
        "radius": radius,
        
    }
    
    # API call
    response = requests.get(foursquare_url, headers=headers, params=params)
    if response.status_code != 200:
        print(f"Error: Foursquare API returned status code {response.status_code}")
        return None
    
    return response.json()
response = query_foursquare(lat, lon)
print(response)

{'results': [{'fsq_id': '4e8df4ccbe7bc43c1895944d', 'categories': [{'id': 17062, 'name': 'Chocolate Store', 'short_name': 'Chocolate Store', 'plural_name': 'Chocolate Stores', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/candystore_', 'suffix': '.png'}}, {'id': 17069, 'name': 'Grocery Store', 'short_name': 'Grocery Store', 'plural_name': 'Grocery Stores', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'LikelyOpen', 'distance': 489, 'geocodes': {'main': {'latitude': 49.269893, 'longitude': -123.095842}, 'roof': {'latitude': 49.269893, 'longitude': -123.095842}}, 'link': '/v3/places/4e8df4ccbe7bc43c1895944d', 'location': {'address': '413 Industrial Ave', 'address_extended': '# 401', 'country': 'CA', 'cross_street': 'at Station St', 'formatted_address': '413 Industrial Ave (at Station St), Vancouver BC V6A 2P8', 'locality': 'Vancouver', 'postcode': 'V6A 2P8', 'region': 'BC'}, 'name': 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [114]:
# Function to parse the response
def parse_foursquare_response(response):
    if not response or "results" not in response:
        return []
    
    poi_list = []
    for poi in response["results"]:
        poi_list.append({
            "POI Name": poi.get("name"),
            "Category": poi.get("categories", [{}])[0].get("name", "Unknown"),
            "Address": poi.get("location", {}).get("address", "N/A"),
            "Latitude": poi.get("geocodes", {}).get("main", {}).get("latitude"),
            "Longitude": poi.get("geocodes", {}).get("main", {}).get("longitude")
        })
    return poi_list

# An example of parsing the response
parsed_data = parse_foursquare_response(response)
print(parsed_data)  


[{'POI Name': 'Beta5 Chocolates', 'Category': 'Chocolate Store', 'Address': '413 Industrial Ave', 'Latitude': 49.269893, 'Longitude': -123.095842}, {'POI Name': "Finch's Market", 'Category': 'Coffee Shop', 'Address': '501 Georgia St E', 'Latitude': 49.278582, 'Longitude': -123.093335}, {'POI Name': 'Pizzeria Farina', 'Category': 'Pizzeria', 'Address': '915 Main St', 'Latitude': 49.276561, 'Longitude': -123.10014}, {'POI Name': 'Harvest Community Foods', 'Category': 'Gourmet Store', 'Address': '243 Union St', 'Latitude': 49.27776, 'Longitude': -123.098052}, {'POI Name': 'The Boxcar', 'Category': 'Arts and Entertainment', 'Address': '917 Main St', 'Latitude': 49.276469, 'Longitude': -123.100214}, {'POI Name': 'Starbucks', 'Category': 'Coffee Shop', 'Address': '753 Terminal Ave', 'Latitude': 49.271053, 'Longitude': -123.087585}, {'POI Name': 'Phnom Penh', 'Category': 'Cambodian Restaurant', 'Address': '244 E. Georgia St', 'Latitude': 49.278403, 'Longitude': -123.098188}, {'POI Name': 'The

Put your parsed results into a DataFrame

In [116]:
# Function to create a DataFrame from parsed data
def create_foursquare_dataframe(parsed_data):
    df = pd.DataFrame(parsed_data)
    return df


foursquare_df = create_foursquare_dataframe(parsed_data)
print(foursquare_df.head())


                  POI Name                Category             Address  \
0         Beta5 Chocolates         Chocolate Store  413 Industrial Ave   
1           Finch's Market             Coffee Shop    501 Georgia St E   
2          Pizzeria Farina                Pizzeria         915 Main St   
3  Harvest Community Foods           Gourmet Store        243 Union St   
4               The Boxcar  Arts and Entertainment         917 Main St   

    Latitude   Longitude  
0  49.269893 -123.095842  
1  49.278582 -123.093335  
2  49.276561 -123.100140  
3  49.277760 -123.098052  
4  49.276469 -123.100214  


In [117]:
foursquare_df.to_csv('foursquare_data.csv', index=False)  # Save as CSV file

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [120]:

# Yelp API info
yelp_api_key = "yUATE8j_Qww667hJIfGLPW_l4W5AnSjZD7Pi1b8YspplmSVAQgR2Qa5hwPgKtVP3N0XoT9FOxymsVMQb61oU_l1tKcmzxapo8-VhKMWGC702yPO2UF7zXJ7_oK86Z3Yx"
yelp_url = "https://api.yelp.com/v3/businesses/search"

# Function to send a request for a single bike station
def query_yelp(lat, lon, category="restaurants", radius=1000):
    headers = {"Authorization": f"Bearer {yelp_api_key}"}
    params = {
        "latitude": lat,
        "longitude": lon,
        "categories": category,
        "radius": radius,
        "limit": 50
    }
    
    response = requests.get(yelp_url, headers=headers, params=params)
    if response.status_code != 200:
        print(f"Error: Yelp API returned status code {response.status_code}")
        print("Details:", response.text)
        return None
    return response.json()



response = query_yelp(lat, lon)
print(response)


{'businesses': [{'id': 'yeNenSjz_HCqngGFU5d8NQ', 'alias': 'phnom-penh-vancouver', 'name': 'Phnom Penh', 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/XzbN2-p09hEFJe2sT30B3g/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/phnom-penh-vancouver?adjust_creative=I3Ht-Uw_IPjv0KelA9NNcA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=I3Ht-Uw_IPjv0KelA9NNcA', 'review_count': 1795, 'categories': [{'alias': 'cambodian', 'title': 'Cambodian'}, {'alias': 'vietnamese', 'title': 'Vietnamese'}], 'rating': 4.2, 'coordinates': {'latitude': 49.278431, 'longitude': -123.098205}, 'transactions': [], 'price': '$$', 'location': {'address1': '244 E Georgia Street', 'address2': '', 'address3': '', 'city': 'Vancouver', 'zip_code': 'V6A 1Z7', 'country': 'CA', 'state': 'BC', 'display_address': ['244 E Georgia Street', 'Vancouver, BC V6A 1Z7', 'Canada']}, 'phone': '+16046825777', 'display_phone': '+1 604-682-5777', 'distance': 652.660730350863, 'business_hours': [{'open'

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [122]:
def parse_yelp_response(response):
    if not response or "businesses" not in response:
        return []
    
    poi_list = []
    for business in response["businesses"]:
        poi_list.append({
            "POI Name": business.get("name"),
            "Category": business.get("categories", [{}])[0].get("title", "Unknown"),
            "Address": business.get("location", {}).get("address1", "N/A"),
            "City": business.get("location", {}).get("city", "N/A"),
            "Rating": business.get("rating", None),
            "Review Count": business.get("review_count", None),
            "Latitude": business.get("coordinates", {}).get("latitude"),
            "Longitude": business.get("coordinates", {}).get("longitude")
        })
    return poi_list

parsed_data = parse_yelp_response(response)
print(parsed_data) 

[{'POI Name': 'Phnom Penh', 'Category': 'Cambodian', 'Address': '244 E Georgia Street', 'City': 'Vancouver', 'Rating': 4.2, 'Review Count': 1795, 'Latitude': 49.278431, 'Longitude': -123.098205}, {'POI Name': 'PiDGiN', 'Category': 'Japanese', 'Address': '350 Carrall Street', 'City': 'Vancouver', 'Rating': 4.0, 'Review Count': 345, 'Latitude': 49.2819071402452, 'Longitude': -123.104188653967}, {'POI Name': 'Di Beppe', 'Category': 'Italian', 'Address': '8 W Cordova Street', 'City': 'Vancouver', 'Rating': 4.2, 'Review Count': 164, 'Latitude': 49.28237705581659, 'Longitude': -123.10450043738042}, {'POI Name': 'Bodega On Main', 'Category': 'Spanish', 'Address': '1014 Main Street', 'City': 'Vancouver', 'Rating': 3.8, 'Review Count': 231, 'Latitude': 49.27548853708847, 'Longitude': -123.09968391901442}, {'POI Name': 'Kissa Tanto', 'Category': 'Japanese', 'Address': '263 E Pender Street', 'City': 'Vancouver', 'Rating': 4.1, 'Review Count': 313, 'Latitude': 49.2806222, 'Longitude': -123.0981135

Put your parsed results into a DataFrame

In [124]:

# function to create dataframe from parsed data
def create_yelp_dataframe(parsed_data):
    df = pd.DataFrame(parsed_data)
    return df


yelp_df = create_yelp_dataframe(parsed_data)
print(yelp_df.head())

         POI Name   Category               Address       City  Rating  \
0      Phnom Penh  Cambodian  244 E Georgia Street  Vancouver     4.2   
1          PiDGiN   Japanese    350 Carrall Street  Vancouver     4.0   
2        Di Beppe    Italian    8 W Cordova Street  Vancouver     4.2   
3  Bodega On Main    Spanish      1014 Main Street  Vancouver     3.8   
4     Kissa Tanto   Japanese   263 E Pender Street  Vancouver     4.1   

   Review Count   Latitude   Longitude  
0          1795  49.278431 -123.098205  
1           345  49.281907 -123.104189  
2           164  49.282377 -123.104500  
3           231  49.275489 -123.099684  
4           313  49.280622 -123.098114  


In [125]:
yelp_df.to_csv('yelp_data.csv', index=False)  # Save as CSV file

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

In [158]:
# Yelp provide more information such as category of the restaurants, rating and review count of them which makes it easier to analyze and understand the data comprehensively

Get the top 10 restaurants according to their rating

In [149]:
yelp_df.sort_values('Rating', ascending=False).head(10)


,POI Name,Category,Address,City,Rating,Review Count,Latitude,Longitude
46,The Garden Strathcona,Cafes,868 E Hastings Street,Vancouver,4.7,42,49.280760,-123.085630
38,Tekkaba,Sushi Bars,68 E Cordova Street,Vancouver,4.6,28,49.282196,-123.102732
27,Blaze Gourmet Burgers,Burgers,43 E 5th Avenue,Vancouver,4.6,36,49.266709,-123.103814
42,Addah,Himalayan/Nepalese,555 Gore Avenue,Vancouver,4.6,17,49.279690,-123.097520
28,Sooda Korean BBQ,Korean,60 West Cordova Street,Vancouver,4.6,43,49.282678,-123.106049
39,The Hide Out Diner,Breakfast & Brunch,101-1951 Glen Drive,Vancouver,4.5,19,49.268890,-123.081184
5,Kozak Ukrainian Restaurant,Ukrainian,1 West Cordova Street,Vancouver,4.5,76,49.282655,-123.104656
7,Oyster Express,Seafood,296 Keefer Street,Vancouver,4.5,171,49.279278,-123.097397
8,Zarak By Afghan Kitchen,Afghan,2102 Main Street,Vancouver,4.5,123,49.266224,-123.100609
41,Bali Thai,Indonesian,88 W Pender Street,Vancouver,4.5,101,49.280271,-123.106227
